In [10]:
# Allows import from parent directory.
import sys

sys.path.append('..')

import json
import os

import numpy as np
from src.providentia_utils import match_perspective, parse_perspectives


In [46]:
# Fix labels with calculating box3d from box3d_projected and perspective.

labels_dir = '../data/r0_a9_dataset/labels'
fixed_labels_dir = '../data/r0_a9_dataset/labels_fixed'
perspectives_dir = '../data/profusion_r0_dataset/r0_s1/05_calibration'

perspectives = parse_perspectives(perspectives_dir)

os.makedirs(fixed_labels_dir, exist_ok=True)

for name in os.listdir(labels_dir):
    with open(os.path.join(labels_dir, name)) as f:
        data = json.load(f)

    perspective = match_perspective(perspectives, data)

    for label in data['labels']:
        image_points = np.array(list(label['box3d_projected'].values())[:4]).T
        image_points[0] *= 1920
        image_points[1] *= 1200
        ground_points = perspective.project_to_ground(image_points)

        x, y = ground_points[:2, :4].mean(1)
        length = np.abs((ground_points[0, [0, 3]] - ground_points[0, [1, 2]]).mean())
        width = np.abs((ground_points[1, [0, 1]] - ground_points[1, [2, 3]]).mean())

        label['box3d']['location'].update(dict(x=x, y=y))
        label['box3d']['dimension'].update(dict(length=length, width=width))

    with open(os.path.join(fixed_labels_dir, name), 'w') as f:
        json.dump(data, f)
